# 0. Initialize

## 0.1. Import Libraries

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os, sys, glob
import gzip
import random
import tqdm
import json
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
from sklearn.model_selection import RepeatedKFold
from IPython import display
import matplotlib as mpl
from matplotlib import pyplot as plt
from google.colab import drive
drive.mount('/content/drive/') 

ModuleNotFoundError: No module named 'google'

## 0.2. DEFINE VARIABLES 

In [32]:
DATA_PATH = '/content/drive/MyDrive/CS412/DataFiles/' # '<insert-your-training-data-path-here>'

ROUND = 3 # This project will have 3 rounds of predictions: 1,2,3
STUDENT_ID_1 = '26617'#'<insert-your-id-here>'
PROJECT_CODE_1 = 'CS4121fa0ea9f0335'


## 0.3. Read Training & Evaluation Data

### 0.3.1. Get the labels for tweets

In [33]:
#trainingTweetDf = pd.read_csv('{}training-tweet.csv'.format(DATA_PATH))
#given data
trainingTweetDf = pd.read_csv('{}training-tweet.csv'.format(DATA_PATH), dtype={'tweet_id': str, 'isPolitical': str})

ydelipinar_training_tweet = pd.read_csv(('{}annotated_tweets_'+STUDENT_ID_1+'.csv').format(DATA_PATH), dtype={'Unnamed: 0': str, 'isPolitical': str})
ydelipinar_training_tweet.drop(['url', 'sentiment', 'isExperiential', 'isInsult', 'topics'], 1, inplace=True)
ydelipinar_training_tweet.rename(columns = {'Unnamed: 0':'tweet_id'}, inplace=True)




# Concatenating -> droping duplicates -> reindexing
trainingTweetDf = pd.concat([ydelipinar_training_tweet, trainingTweetDf], axis='rows')


#trainingTweetDf.drop_duplicates(subset=['tweet_id'], inplace=True)
#trainingTweetDf.reset_index(drop=True, inplace=True)
trainingTweetDf

<ipython-input-33-ebaa08c48230>:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  ydelipinar_training_tweet.drop(['url', 'sentiment', 'isExperiential', 'isInsult', 'topics'], 1, inplace=True)
<ipython-input-33-ebaa08c48230>:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  murat_training_tweet.drop(['url', 'sentiment', 'isExperiential', 'isInsult', 'topics'], 1, inplace=True)
<ipython-input-33-ebaa08c48230>:15: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  cihan_training_tweet.drop(['url', 'sentiment', 'isExperiential', 'isInsult', 'topics'], 1, inplace=True)


,tweet_id,isPolitical
0,1595468724127105024,Yes
1,1551163840368414722,No
2,1596944322687303680,Yes
3,1554863935928541188,Yes
4,1595778348143284225,Yes
...,...,...
2995,1593539327623151619,Yes
2996,1393886554062524418,No
2997,1597925615092764672,Yes
2998,1585291418616176640,Yes


In [34]:
trainingTweetDf.isPolitical.value_counts()

Yes    2149
No     1051
Name: isPolitical, dtype: int64

### 0.3.2. Get the labels for users

In [35]:
trainingUserDf = pd.read_csv('{}training-user.csv'.format(DATA_PATH))

# My annotations
ydelipinar_training_user = pd.read_csv(('{}annotated_users_'+STUDENT_ID_1+'.csv').format(DATA_PATH), dtype={'Unnamed: 0': str, 'isBot': str})
ydelipinar_training_user.drop(['url', 'isOrganizational', 'gender', 'isTroll'], 1, inplace=True)
ydelipinar_training_user.rename(columns = {'Unnamed: 0':'screen_name'}, inplace=True)


# Concatenating -> droping duplicates -> reindexing
trainingUserDf = pd.concat([ydelipinar_training_user, trainingUserDf], axis='rows')



# trainingUserDf.drop_duplicates(subset=['screen_name'], inplace=True)
# trainingUserDf.reset_index(drop=True, inplace=True)
trainingUserDf


<ipython-input-35-9d1286b5f702>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  ydelipinar_training_user.drop(['url', 'isOrganizational', 'gender', 'isTroll'], 1, inplace=True)
<ipython-input-35-9d1286b5f702>:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  murat_training_user.drop(['url', 'isOrganizational', 'gender', 'isTroll'], 1, inplace=True)
<ipython-input-35-9d1286b5f702>:13: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  cihan_training_user.drop(['url', 'isOrganizational', 'gender', 'isTroll'], 1, inplace=True)


,screen_name,isBot
0,koftecancaddy,No
1,ahaber,No
2,selahat03949652,No
3,erdin06357062,No
4,bhct__necatii,No
...,...,...
2995,djblumenberg,No
2996,mel1sq,No
2997,eren_yz1,Yes
2998,ergnyildiz4,No


In [36]:
trainingUserDf.isBot.value_counts()

No     2424
Yes     576
Name: isBot, dtype: int64

### 0.3.3. Expand your dataset with metadata and tweets

In [37]:
# You can also expand training data by downloading your own labeled datasets following the link
# Download the documents under "Link to training data"

print('http://www.onurvarol.com/Annotation-CS412-202201/reports/report_{}.html'.format(PROJECT_CODE_3))

http://www.onurvarol.com/Annotation-CS412-202201/reports/report_CS412bec5219f45cd.html


# 1. EXTRACT FEATURES
Under *1.1. Political Tweet Detection* and *1.2. Bot Detection*, we firstly collect raw data for processing. We then combine some of them (total_interactions = num_favorites + num_retweets) or use them to extract features (whether the tweet has one of the political entities @meralaksener, @kilicdarogluk etc.).

We expect you to collect more raw data from **tweet_metadata**, **user_profiles** and **user_tweets** files by creating a function as shown in below examples such as *check_if_retweet()* and using it while iterating over data as shown under *Merge Collected Features*.

We also expect you to create new variables as much as you can from the data in order to make your predictions more accurate. For example, you may want to check:

- The tweet sources that a user frequently uses
- Whether the user is a verified account or not

...

to assess whether **a user is a bot or not** and whether **a tweet is political or not**.

In [38]:
PATH_TO_DOWNLOADED = DATA_PATH # 

## 1.1. Political Tweet Detection
This part stands for the feature extraction of tweets. We start with collecting the raw data from *tweet_metadata*, then use some of them to extract features.

### 1.1.1. Get Raw Data

#### 1.1.1.1. Check if Retweet

In [39]:
def check_if_retweet(tweet_metadata_line):
    is_retweet = 0
    retweeted_username = None

    try:
        tweet_metadata_line['retweeted_status']
        retweeted_username = tweet_metadata_line['retweeted_status']['user']['screen_name'].lower()
        is_retweet = 1

    except KeyError:
        pass

    return is_retweet, retweeted_username

#### 1.1.1.2. Get Tweet Text

In [40]:
def get_tweet_text(tweet_metadata_line):
    text = tweet_metadata_line['text']
    
    return text

#### 1.1.1.3. Get Tweet ID

In [41]:
def get_tweet_id(tweet_metadata_line):
    id_str = tweet_metadata_line['id_str']
    
    return id_str

#### 1.1.1.4. Get Number of Mentions and Hashtags

In [42]:
def get_number_mentions_hashtags(tweet_metadata_line):
    num_mentions = len(tweet_metadata_line['entities']['user_mentions'])
    num_hashtags = len(tweet_metadata_line['entities']['hashtags'])

    return num_mentions, num_hashtags

#### 1.1.1.5. Get Number of Retweets and Favorites

In [43]:
def get_number_retweets_favorites(tweet_metadata_line):
    retweet_count = tweet_metadata_line['retweet_count']
    favorite_count = tweet_metadata_line['favorite_count']
    
    return retweet_count, favorite_count

#### 1.1.1.6. Get User Info

In [44]:
def get_user_info(tweet_metadata_line):
    id = tweet_metadata_line['user']['id_str']
    screen_name = tweet_metadata_line['user']['screen_name'].lower()
    description = tweet_metadata_line['user']['description']

    return id, screen_name, description

### 1.1.2. Derive Manually Crafted Features

#### 1.1.2.1. Check for political entity in text

In [45]:
def check_description(text):

  desc_entities = [ 'akp', 'chp', 'hdp', 'mhp','ak parti', 'iyi part','akparti','iyi parti' ,'parti' 'RTErdogan', 'vatan', 'sevdal','devlet',
                    'TÜRKİYE', 'reis', 'recep tayip' , 'Erdoğan', 'Cumhur İttifakı', 'Kemalist', 'eyt', 'Başkan', 'siyas', 'laik', 'Atama', 'TÜRK', '🇹🇷']


  entities_in_text = [ent for ent in desc_entities if ent.lower() in text.lower()]

  return len(entities_in_text)

In [46]:
def check_link(text):

  if('https://t.co' in text):
    return 1
  else:
    return 0

In [47]:
def check_political_ent(text):
    text = text.lower()

  
    # Possible keywords that can be found in political tweets.
    tweet_entities = [ 'akp', 'Akp','chp','CHP', 'ChP' 'hdp', 'mhp', 'iyi part','iyiparti','akparti','ak parti' ,'RTErdogan','suleymansoylu','Süleyman Soylu', 'TC_icisleri','drfahrettinkoca' , 'ekrem_imamoglu','ekrem imamoğlu','Ekrem İmamoğlu', 'imamoğlu', 'MevlutCavusoglu','meral_aksener','Akşener','kilicdarogluk','Canan_Kaftanci '
                      'Kemal Kılıçdaroğlu', 'mansuryavas06','06melihgokcek','tcmeb','umitozdag', 'Ümit Özdağ','tcailesosyal','devletbahceli','Bahçeli','alibabacan', 'NureddinNebati' , 'başkan', 'reis', 'bybekirbozdag', 'hdpdemirtas','pkk', 'Erdoğan','Erdogan','Recep Tayyip Erdoğan', 'recep tayyip','RecepTayyipErdoğan' 
                      'Tayyip', 'Başkanımız', "Sayın",'Belediye', 'Afgan', 'suriye', 'devlet','Seçim', 'Cumhurbaşkan','atama', "kyk", 'eyt','meclis', 'vedatbilgn', 'bekliyor','Sosyal Hizmet','EmineErdogan' , 'nato', 'TRT', 'vatan', 'Babacan', 'Cumhuriyet','vali', 'DpGultekinUysal', 'GultekinUysal', 
                      'asgari','syn', 'tcbestepe', 'Türkiye', 'MSTanrikulu', 'Tanrıkulu','bekliyor','istiyoruz', 'TERÖR', 'teror', 'sevici',  'Ahaber', 'vekil',  'Atanmak', 'vekilince','akaraismailoglu' , 'engelli','Silivri', 'adalet', 'parti', 'istihdam', 'MAĞDUR', 'aday', 'altılı masa','barisatay',
                      'atan', 'Cumhuriyet', 'talimat', "Umit_ozdag", 'Diyanet', 'Özal', 'yargi', 'yargı', 'KAYYUM', 'macron', 'Referandum', 'abd','putin', 'Kadir Topbaş', 'bakan', 'Mansur', 'Canan_Kaftanci', 'Ekonomi', 'nagihan', 'Cumhuriyet Halk Partisi', 'taleb', 'talep', 'personel', 'tunç soyer', 'Mansur Yavaş'
                      'arap', 'Ülke', '🇹🇷', 'kamu', 'dfikrisaglar', 'tele1comtr','alimahir', 'arzuerdemDB','AyhanBarut01','VahitKirisci' ,'dfikrisaglar', 'anayasa', 'MUHSİN YAZİCIOĞLU', 'Mustafa_Destici', 'Hekim','varank', 'alpayozalan35', 'bulent_masaoglu', 'CihanBasaranMM', 'LutfuTurkkan', 'celebimehmeta','Ahmet_Davutoglu',
                       'Kürd', 'mahirunal', 'halktvcomtr', 'AvOzlemZengin', 'eczozgurozel', 'TCAytunCiray','ibrahim_ozkan61', 'gurseltekin34', 'LutfuTurkkan', 'enginozkoc', 'kamudanhaber', 'enginozkoc', 'faikoztrak', 'ecevit', 'sgirgin48tbmm', 'ayhanbilgen', 'mustafaergen','AvSerkanBayram', 'AysuBankoglu', 'SezaiTemelli', 
                      's_turpcu', 'PARTi', 'yunanistan', 'Atatürk','apo', 'samiltayyar27', 'yavuzagiraliog', 'Kılıçdaroğlu','Utan', 'corasalih', 'fethigurer','murat_kurum' , 'oerdem42', 'yavuzagiraliog', 'erenerdemnet', 'SekibAvdagic', 'RHisarciklioglu', 'haluktatar', 'KHK', 'fahrettinaltun', 'seferaycan', 'Taşeron', 
                      'ayyildirim1', 'leylasahinusta', 'oznurcalik', 'erkankandemir', 'solcugazete', 'Kontenjan', 'gergerliogluof', 'TurkishIndy', 'NumanKurtulmus', 'DrVeyselTiryaki', 'Rusiya', 'rusya', 'ukrayna', 'efkanala', 'SirakayaZafer', 'HsnBozkurt', 'add_genelmerkez', 'oznurcalik', 'ittifak', 'akbasogluemin',
                      'meraldanis', 'yusufhalacoglu','yenisafak', 'akadirkaraduman', 'Av_Burcu03', 'mkulunk', 'BuroMemurSen_', 'huseyinozturk_1', 'bedelli', 'uzun adam', 'oguzksalici', 'biden', 'enflasyon', 'gazetesozcu', 'MetinErgun48', 'SevdaGulTuncer'
                      
                      'RtErdogan','kiliCdarogluk','meRal_aksener','meliHgokcek', 'suleymansoyLu', 'ekrem_imamogLu','İmamoğlu', 'umiTozdag', ]
                
 
    entities_in_text = [ent for ent in tweet_entities if ent.lower() in text.lower()]

    return len(entities_in_text)


#### 1.1.2.2. Number of total interactions

In [48]:
def total_interactions(retweet_count, favorite_count):
    total_num_interactions = retweet_count + favorite_count
    
    return total_num_interactions

####1.1.2.3 Number of Total Political Word ratio


In [49]:
def political_word_total_word_ratio(text, num):
    num_political_entities = num
    total_word = len(set(text.split()))

    return num_political_entities/total_word

### 1.1.2. Collect data using the functions above and transform into a Pandas DataFrame

In [50]:
dfPolitical = {'tweet_id':[],
              'is_retweet':[],
              'retweeted_username':[],
              'user_description':[], 
              'text':[],
              'num_mentions':[],
              'num_hashtags':[],
              'num_retweets':[],
              'num_favorites':[],
              'user_id':[],
              'user_screen_name':[],
              'num_political_entities_tweet':[],
              'num_political_entities_description': [],
              'has_link': [],
              'political_word_ratio': [],             
              'total_interactions':[]}


with gzip.open(f"{PATH_TO_DOWNLOADED}tweet_metadata.jsons.gz", "rb") as f:
    for line in f:
        line = json.loads(line)
        
        # raw data:
        id_str = get_tweet_id(line)
        is_retweet, retweeted_username = check_if_retweet(line)
        text = get_tweet_text(line)
        num_mentions, num_hashtags = get_number_mentions_hashtags(line)
        retweet_count, favorite_count = get_number_retweets_favorites(line)
        user_id_str, screen_name, user_description = get_user_info(line)
  
        # manually crafted data:
        total_num_interactions = total_interactions(retweet_count, favorite_count)
        num_political_entities_tweet = check_political_ent(text)
        num_political_entities_in_description = check_description(user_description)
        political_word_ratio = political_word_total_word_ratio(text, num_political_entities_tweet)
        has_link = bool(check_link(text))

        dfPolitical['tweet_id'].append(id_str)
        dfPolitical['is_retweet'].append(is_retweet)
        dfPolitical['retweeted_username'].append(retweeted_username)
        dfPolitical['text'].append(text)
        dfPolitical['num_mentions'].append(num_mentions)
        dfPolitical['num_hashtags'].append(num_hashtags)
        dfPolitical['num_retweets'].append(retweet_count)
        dfPolitical['num_favorites'].append(favorite_count)
        dfPolitical['user_id'].append(user_id_str)
        dfPolitical['user_screen_name'].append(screen_name)
        dfPolitical['user_description'].append(user_description)
        dfPolitical['total_interactions'].append(total_num_interactions)
        dfPolitical['num_political_entities_tweet'].append(num_political_entities_tweet)
        dfPolitical['num_political_entities_description'].append(num_political_entities_in_description)
        dfPolitical['has_link'].append(has_link)
        dfPolitical['political_word_ratio'].append(political_word_ratio)

In [51]:
dfPolitical = pd.DataFrame(dfPolitical)
dfPolitical

,tweet_id,is_retweet,retweeted_username,user_description,text,num_mentions,num_hashtags,num_retweets,num_favorites,user_id,user_screen_name,num_political_entities_tweet,num_political_entities_description,has_link,political_word_ratio,total_interactions
0,1588568792984346624,0,None,shu/\nburaya afilli bir söz yazdığımı varsayın,"Sosyal Hizmetin temelini çocuk oluşturur,çocuğ...",0,0,49,98,920963718103650304,maviruh_,1,0,True,0.083333,147
1,1588452263047069697,0,None,,"@mahirunal Gavur İzmir ya onlar, hani Cumhuriy...",1,0,0,0,595514060,mtfdan,3,0,False,0.230769,0
2,1569589330544398336,0,None,Vatan Sevdalisi,#ŞehitAdayıUzmÇvşaKadro\nSiz İstesenizde Istem...,0,1,0,0,1356375754561490947,ahsucilginuzman,1,2,True,0.071429,0
3,1570428119609139201,0,None,HighOne,@ajans_muhbir Siz kaypak olmayıp onay vermesey...,1,0,0,0,1478775431008595968,hamitelkelle,1,0,True,0.058824,0
4,1551163840368414722,0,None,,Engelli öğretmenler olarak önümüzdeki engeller...,0,0,0,0,1511976696337113088,sed58417690,3,0,True,0.272727,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33528,1568595408233832448,0,None,RT ONAYLADIĞIM ANLAMINA GELMEZ\nArtık fikirler...,Gerçek kimlik taşımayan hesaplara cevap vermem...,0,0,9,81,576247173,ardanzenturk,0,0,True,0.000000,90
33529,1584027427696959488,0,None,Bridge design engineer Yaay hesabı : dilfiruz,@umitozdag Neden Suriyelilerle ilgili bu kadar...,1,0,1,8,162308585,ozgul_61,5,0,True,0.277778,9
33530,1585945783307730945,0,None,,@celebimehmeta Niye Türkiye yüzyılıda.Türkiye ...,1,0,0,1,415025519,ladrekova,4,0,False,0.571429,1
33531,1569748909521801221,1,muazzezeralp,,RT @muazzezeralp: @Doan58213655 @denizkonur @N...,7,1,6,0,1442125177727307781,yapikytgrivrlsn,6,0,False,0.545455,6


## 1.2. From Users

### 1.2.1. Get user metadata from user_profiles.jsons.gz

#### 1.2.1.1. Get user info metadata

In [52]:
def get_user_info_metadata(user_metadata_line):
    
    user_id = user_metadata_line['id_str']
    user_name = user_metadata_line['name']
    user_screen_name = user_metadata_line['screen_name'].lower()
    user_location = user_metadata_line['location']
    user_description = user_metadata_line['description']
    user_followers_count = user_metadata_line['followers_count']
    user_friends_count = user_metadata_line['friends_count']
    user_is_verified = user_metadata_line['verified']
    user_has_default_photo = user_metadata_line['default_profile_image']
    user_statuses_count = user_metadata_line['statuses_count']
    
    dictionary = {'user_id':user_id, 'user_name': user_name, 'user_screen_name':user_screen_name, 'user_location':user_location,
     'user_description':user_description, 'user_followers_count':user_followers_count, 'user_friends_count':user_friends_count,
     'user_statuses_count': user_statuses_count, 'user_is_verified':user_is_verified, 'user_has_default_photo': user_has_default_photo}

    return dictionary

#### 1.2.1.2. Get followers/(followers+friends) ratio

In [53]:
def get_followers_all_ratio(user_followers_count, user_friends_count):
    
    if user_friends_count + user_followers_count == 0:
        followers_all_ratio = 0

    else:
        followers_all_ratio =  user_followers_count / (user_friends_count + user_followers_count)

    return followers_all_ratio

#### 1.2.1.3. Get description length

In [54]:
def get_desc_len(user_description):
    
    description_len = len(user_description)

    return description_len

In [55]:
dfBot = {'user_id':[],
         'user_name':[],
         'user_screen_name':[],
         'user_location':[],
         'user_description':[],
         'user_followers_count':[],
         'user_friends_count':[],
         'description_len':[],
         'followers_to_all_ratio':[],
         'user_is_verified':[],
         'user_has_default_photo': [],
         'user_statuses_count':[]
         }

with gzip.open(f"{PATH_TO_DOWNLOADED}user_profiles.jsons.gz", "rb") as f:
    for line in f:
        line = json.loads(line)

        dictionary = get_user_info_metadata(line)
        for k,v in dictionary.items():
            dfBot[k].append(v)

        
        # manually crafted data:
        description_len = get_desc_len(dictionary['user_description'])
        dfBot['description_len'].append(description_len)
        
        followers_all_ratio = get_followers_all_ratio(dictionary['user_followers_count'], 
                                                      dictionary['user_friends_count'])

        dfBot['followers_to_all_ratio'].append(followers_all_ratio)

In [56]:
dfBot = pd.DataFrame(dfBot)
dfBot

,user_id,user_name,user_screen_name,user_location,user_description,user_followers_count,user_friends_count,description_len,followers_to_all_ratio,user_is_verified,user_has_default_photo,user_statuses_count
0,1431241870848450577,Nasreena Khan Wazir,nasreenakhan006,"Islamabad, Pakistan",Student,65,185,7,0.260000,False,False,2551
1,1304340303080386560,fania :((((,scorpiehoez,bogor,have a holly jolly🎄,8235,3011,19,0.732260,False,False,42771
2,1116042038577958914,Yusuf Aksoy,yusufak63712920,,"Bir şeyden pişmanlık duymak istemiyorsan,her ş...",95,399,64,0.192308,False,False,14300
3,4859899931,Be (VIXX6) ama oppalarının düğününe gidemiyor,nedenburdaysam,Hufflepuff ortak salon,"SMStan\n/St☆rlight ///come on girls,this is ou...",40,83,65,0.325203,False,False,21303
4,2225373636,SLMDMR,biologselim,,BİYOLOG🔬🦠\nNanoteknoloji,100,98,23,0.505051,False,False,1629
...,...,...,...,...,...,...,...,...,...,...,...,...
29665,1320834618220781569,islammm,islam_mrsj,,,229,217,0,0.513453,False,False,2396
29666,111074128,Melda Onur,meldaonur,,"Şekersiz çay, etsiz sofra, SAVAŞSIZ dünya... 🐌...",212457,5428,100,0.975088,False,False,75178
29667,36946875,ali ydm,ali_ydm,"İstanbul, Türkiye",hayat oyunsa bende jeton çok,116,141,28,0.451362,False,False,6482
29668,2389587396,Türkan Usta,turkanusta,"Ankara, Türkiye",Ustaya sormuşlar; hayatta yaptığın en büyük is...,1669,3639,116,0.314431,False,False,121113


### 1.2.2. Get Tweet Info of Users in user_profiles.jsons.gz

#### 1.2.2.1. Check ratio of retweets to all tweets

In [57]:
def get_retweet_tweet_ratio(line):
    number_retweets = 0
    number_original_tweets = 0

    for tweet in line['tweets']:
        try:
            tweet['retweeted_status']
            number_retweets += 1
                
        except:
            number_original_tweets += 1
            
    total_tweets = number_retweets + number_original_tweets
    
    if total_tweets == 0:
        retweet_total_ratio = None
    else:
        retweet_total_ratio = number_retweets/(total_tweets)
    
    return retweet_total_ratio


#### 1.2.2.2. Check median number of favorites

In [58]:
def get_median_number_favorites(line):
    num_median_favorites = np.median([tweet['favorite_count'] for tweet in line['tweets']])

    return num_median_favorites

### 1.2.3. Collect data using the functions above and transform into a Pandas DataFrame

In [59]:
dfBotTweets = {'user_id':[],
               'retweet_total_ratio':[],
               'num_median_favorites':[],
               'num_of_tweets':[]     
              }

i = 0

with gzip.open(f"{PATH_TO_DOWNLOADED}user_tweets.jsons.gz", "rb") as f:
    for line in f:

        line = json.loads(line)

        user_id = line['user_id']
        dfBotTweets['user_id'].append(user_id)
        
        retweet_total_ratio = get_retweet_tweet_ratio(line)
        dfBotTweets['retweet_total_ratio'].append(retweet_total_ratio)
        
        num_median_favorites = get_median_number_favorites(line)
        dfBotTweets['num_median_favorites'].append(num_median_favorites)
        
        dfBotTweets['num_of_tweets'].append(len(line['tweets']))


        i += 1
        if i % 1000 == 0:
            print(i)

/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000


In [60]:
dfBotTweets = pd.DataFrame(dfBotTweets)
dfBotTweets

,user_id,retweet_total_ratio,num_median_favorites,num_of_tweets
0,594642154,0.115000,2.0,200
1,525600289,0.005025,1.0,199
2,931895965501534209,0.900000,0.0,200
3,1591543462746329088,0.185000,0.0,200
4,734801354749796352,1.000000,0.0,200
...,...,...,...,...
28310,1591370361488252928,0.800000,0.0,200
28311,1475272459616235525,0.825000,0.0,200
28312,1096753792731750401,0.051020,1.0,196
28313,1269527617687953409,0.095000,2.0,200


### 1.2.3. Merge dfBot and dfBotTweets

In [61]:
dfBotAll = dfBot.merge(dfBotTweets,
                       how='left')

dfBotAll[['retweet_total_ratio', 'num_median_favorites']] = dfBotAll[['retweet_total_ratio', 'num_median_favorites']].fillna(0)

dfBotAll

,user_id,user_name,user_screen_name,user_location,user_description,user_followers_count,user_friends_count,description_len,followers_to_all_ratio,user_is_verified,user_has_default_photo,user_statuses_count,retweet_total_ratio,num_median_favorites,num_of_tweets
0,1431241870848450577,Nasreena Khan Wazir,nasreenakhan006,"Islamabad, Pakistan",Student,65,185,7,0.260000,False,False,2551,0.395939,0.0,197.0
1,1304340303080386560,fania :((((,scorpiehoez,bogor,have a holly jolly🎄,8235,3011,19,0.732260,False,False,42771,0.125000,0.0,200.0
2,1116042038577958914,Yusuf Aksoy,yusufak63712920,,"Bir şeyden pişmanlık duymak istemiyorsan,her ş...",95,399,64,0.192308,False,False,14300,0.910000,0.0,200.0
3,4859899931,Be (VIXX6) ama oppalarının düğününe gidemiyor,nedenburdaysam,Hufflepuff ortak salon,"SMStan\n/St☆rlight ///come on girls,this is ou...",40,83,65,0.325203,False,False,21303,0.015306,1.0,196.0
4,2225373636,SLMDMR,biologselim,,BİYOLOG🔬🦠\nNanoteknoloji,100,98,23,0.505051,False,False,1629,0.659898,0.0,197.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29665,1320834618220781569,islammm,islam_mrsj,,,229,217,0,0.513453,False,False,2396,0.015000,1.0,200.0
29666,111074128,Melda Onur,meldaonur,,"Şekersiz çay, etsiz sofra, SAVAŞSIZ dünya... 🐌...",212457,5428,100,0.975088,False,False,75178,0.291457,2.0,199.0
29667,36946875,ali ydm,ali_ydm,"İstanbul, Türkiye",hayat oyunsa bende jeton çok,116,141,28,0.451362,False,False,6482,0.061538,0.0,195.0
29668,2389587396,Türkan Usta,turkanusta,"Ankara, Türkiye",Ustaya sormuşlar; hayatta yaptığın en büyük is...,1669,3639,116,0.314431,False,False,121113,0.995000,0.0,200.0


# 2. TRAIN MODEL

## 2.1. Political Tweet Prediction

### 2.1.1. Merge dfPolitical data with labels

In [62]:
dfPoliticalAll_train = dfPolitical.merge(trainingTweetDf,
                                         on='tweet_id')

dfPoliticalAll_train

,tweet_id,is_retweet,retweeted_username,user_description,text,num_mentions,num_hashtags,num_retweets,num_favorites,user_id,user_screen_name,num_political_entities_tweet,num_political_entities_description,has_link,political_word_ratio,total_interactions,isPolitical
0,1588568792984346624,0,None,shu/\nburaya afilli bir söz yazdığımı varsayın,"Sosyal Hizmetin temelini çocuk oluşturur,çocuğ...",0,0,49,98,920963718103650304,maviruh_,1,0,True,0.083333,147,Yes
1,1588452263047069697,0,None,,"@mahirunal Gavur İzmir ya onlar, hani Cumhuriy...",1,0,0,0,595514060,mtfdan,3,0,False,0.230769,0,Yes
2,1569589330544398336,0,None,Vatan Sevdalisi,#ŞehitAdayıUzmÇvşaKadro\nSiz İstesenizde Istem...,0,1,0,0,1356375754561490947,ahsucilginuzman,1,2,True,0.071429,0,Yes
3,1570428119609139201,0,None,HighOne,@ajans_muhbir Siz kaypak olmayıp onay vermesey...,1,0,0,0,1478775431008595968,hamitelkelle,1,0,True,0.058824,0,Yes
4,1551163840368414722,0,None,,Engelli öğretmenler olarak önümüzdeki engeller...,0,0,0,0,1511976696337113088,sed58417690,3,0,True,0.272727,0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,1565798698029924353,0,None,,@dbdevletbahceli Şoför,1,0,0,0,1373715344301391882,ahmetazrak8,2,0,False,1.000000,0,No
3196,1586019337248309248,0,None,,Türkiyenin geleceği ücretli öğretmenliğe emane...,4,0,0,0,1583586562817785860,taskinali003,3,0,True,0.272727,0,Yes
3197,1589888339108585474,0,None,,@enginozkoc ARTIK YETER !\nSUÇ DUYURUSUDUR\n@R...,3,0,0,0,1589554549790052353,00can02,6,0,True,0.375000,0,Yes
3198,1587113229041508352,0,None,,@mahirunal Parti adına doğrusunu yaptın.,1,0,0,0,1491090669263278081,teomantiryakio2,3,0,False,0.600000,0,Yes


In [63]:
from collections import Counter
dfPolitical_isPolitic = dfPoliticalAll_train[dfPoliticalAll_train["isPolitical"] == "Yes"]
print(dfPolitical_isPolitic.shape)
Counter(" ".join(dfPolitical_isPolitic["text"]).split()).most_common(50)

(2149, 17)


[('@RTErdogan', 368),
 ('ve', 211),
 ('bir', 191),
 ('Sayın', 142),
 ('bu', 139),
 ('@kilicdarogluk', 131),
 ('@meral_aksener', 116),
 ('için', 112),
 ('de', 81),
 ('olarak', 80),
 ('ile', 79),
 ('atama', 79),
 ('ne', 78),
 ('Bu', 74),
 ('@vedatbilgn', 73),
 ('@drfahrettinkoca', 70),
 ('@umitozdag', 68),
 ('@suleymansoylu', 64),
 ('@ekrem_imamoglu', 61),
 ('da', 56),
 ('çok', 55),
 ('en', 54),
 ('bin', 53),
 ('gibi', 46),
 ('her', 43),
 ('kadar', 43),
 ('kadro', 41),
 ('o', 39),
 ('bekliyor', 39),
 ('yok', 39),
 ('değil', 37),
 ('Ziraat', 36),
 ('sayın', 35),
 ('daha', 35),
 ('istiyoruz', 34),
 ('@Akparti', 34),
 ('Biz', 33),
 ('var', 33),
 ('ama', 33),
 ('siz', 32),
 ('olsun', 32),
 ('mi', 31),
 ('artık', 31),
 ('100', 31),
 ('@akaraismailoglu', 30),
 ('@dbdevletbahceli', 30),
 ('@NureddinNebati', 29),
 ('olmak', 29),
 ('@06melihgokcek', 29),
 ('Engelli', 29)]

### 2.1.2. Separate X and y values
We only use 3 features here to create a baseline model. However, it is not enough to get good results.

In [64]:

X = dfPoliticalAll_train[['num_political_entities_tweet', 'num_political_entities_description', 'num_mentions', 'num_hashtags', 'total_interactions',  'political_word_ratio', 'has_link']]



y = dfPoliticalAll_train['isPolitical'].apply(lambda x: 1 if x=='Yes' else 0)

### 2.1.3. Train - validation split

In [65]:
X.head()

,num_political_entities_tweet,num_political_entities_description,num_mentions,num_hashtags,total_interactions,political_word_ratio,has_link
0,1,0,0,0,147,0.083333,True
1,3,0,1,0,0,0.230769,False
2,1,2,0,1,0,0.071429,True
3,1,0,1,0,0,0.058824,True
4,3,0,0,0,0,0.272727,True


In [66]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=55)

### 2.1.4. Train the model

Here, you may use different models such as neural networks, XGBoost, AdaBoost, RandomForest, Linear Regression, Logistic Regression etc. to see which model does the best. Also, you can use grid_search_cv() or a basic for loop to optimize the hyperparameters of your model.

In [67]:
# Hyper parameter optimization for isPolitical
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, make_scorer
import xgboost as xgb

mse = make_scorer(mean_squared_error, greater_is_better=False)
params = {
    'max_depth': range(3, 10, 2),
    'min_child_weight':range(1,6,2),
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)],
    'gamma':[i/10.0 for i in range(0,5)]
}

# create an XGB Boost instance
xgb_reg = xgb.XGBRegressor(
    objective='binary:logistic'
  
)

# searching the model
politicalModel = GridSearchCV(estimator = xgb_reg, param_grid= params, n_jobs = 4, cv = 5, verbose = True, scoring=mse)

# fitting model
politicalModel.fit(X_train, y_train)

politicalModel.best_estimator_

Fitting 5 folds for each of 960 candidates, totalling 4800 fits


XGBRegressor(colsample_bytree=0.6, gamma=0.3, min_child_weight=5,
             objective='binary:logistic', subsample=0.6)

In [68]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import  mean_squared_error, make_scorer

mse = make_scorer(mean_squared_error, greater_is_better=False)



parameters = { 
    'learning_rate': [0.2, 0.3, 0.4],
    'max_depth': [4,6,8],
    'n_estimators': [50,100,150],
    'min_child_weight':[5,6,7],
    'subsample':[0.4,0.6,0.8,1],
    'colsample_bytree':[0.4,0.6,0.8,1],
    'gamma':[0.3,0.4,0.5],
    'reg_lambda': [0,0.3,0.6,1]
    }



estimator = xgb.XGBRegressor( objective='binary:logistic')

politicalModel1 = GridSearchCV(estimator, parameters, n_jobs=4, 
                   cv=5, 
                   scoring=mse,
                   verbose=2, refit=True)

politicalModel1.fit(X_train, y_train)



Fitting 5 folds for each of 15552 candidates, totalling 77760 fits
[0]	validation_0-error:0.314063
[1]	validation_0-error:0.307813
[2]	validation_0-error:0.19375
[3]	validation_0-error:0.153125
[4]	validation_0-error:0.153125
[5]	validation_0-error:0.15625
[6]	validation_0-error:0.175
[7]	validation_0-error:0.170312
[8]	validation_0-error:0.175
[9]	validation_0-error:0.15625
[10]	validation_0-error:0.157813
[11]	validation_0-error:0.157813
[12]	validation_0-error:0.153125
[13]	validation_0-error:0.15625
[14]	validation_0-error:0.15625
[15]	validation_0-error:0.154687
[16]	validation_0-error:0.153125
[17]	validation_0-error:0.153125
[18]	validation_0-error:0.151562
[19]	validation_0-error:0.15
[20]	validation_0-error:0.15
[21]	validation_0-error:0.15
[22]	validation_0-error:0.15
[23]	validation_0-error:0.15
[24]	validation_0-error:0.15
[25]	validation_0-error:0.15
[26]	validation_0-error:0.15
[27]	validation_0-error:0.15
[28]	validation_0-error:0.15
[29]	validation_0-error:0.15
[30]	val

GridSearchCV(cv=5,
             estimator=XGBRegressor(early_stopping_rounds=10,
                                    objective='binary:logistic'),
             n_jobs=4,
             param_grid={'colsample_bytree': [0.4, 0.6, 0.8, 1],
                         'gamma': [0.3, 0.4, 0.5],
                         'learning_rate': [0.2, 0.3, 0.4],
                         'max_depth': [4, 6, 8], 'min_child_weight': [5, 6, 7],
                         'n_estimators': [50, 100, 150],
                         'reg_lambda': [0, 0.3, 0.6, 1],
                         'subsample': [0.4, 0.6, 0.8, 1]},
             scoring=make_scorer(mean_squared_error, greater_is_better=False),
             verbose=2)

###Making Prediction and displaying the MSE

In [69]:
preds1 = politicalModel1.predict(X_valid)

mse_political1 = mean_squared_error(y_valid, preds1)

print("MSE:", mse_political1)


MSE: 0.11761673055215136


## 2.2. Bot Detection

### 2.2.1. Merge dfBotAll data with labels

In [71]:
dfBotAll.user_screen_name = dfBotAll.user_screen_name.str.lower()

In [72]:
dfBotAll_train = dfBotAll.merge(trainingUserDf,
                               left_on='user_screen_name',
                               right_on='screen_name')

dfBotAll_train

,user_id,user_name,user_screen_name,user_location,user_description,user_followers_count,user_friends_count,description_len,followers_to_all_ratio,user_is_verified,user_has_default_photo,user_statuses_count,retweet_total_ratio,num_median_favorites,num_of_tweets,screen_name,isBot
0,1512081815292432394,sezgin,sezgin953116371,,,46,430,0,0.096639,False,True,1535,0.050251,0.0,199.0,sezgin953116371,No
1,1425452291428077571,Adem Koç,gogoadem61,,,14,171,0,0.075676,False,False,113,0.761062,0.0,113.0,gogoadem61,No
2,328164303,Necmettin Balıkçı,dewil511,,,21,49,0,0.300000,False,False,219,0.010101,0.0,198.0,dewil511,Yes
3,1343666971368431622,Night Bird⁷🦉,midnight__bird,,"La vie est un sommeil, l’amour en est le rêve...",422,260,48,0.618768,False,False,15191,0.085000,1.0,200.0,midnight__bird,No
4,1240932880488038400,Samed Pınarcı,samedpinarci,,Orman Mühendisi - Orman İşletme Şefi - Orman G...,133,202,60,0.397015,False,False,3734,0.780000,0.0,200.0,samedpinarci,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,1286770207134973954,Hamide Arabacı,anka6054,,,151,61,0,0.712264,False,False,5288,0.000000,1.0,200.0,anka6054,No
2996,1598032338323214338,atamabekleyenbahceci,atamabekleyenzz,,,173,367,0,0.320370,False,False,311,0.580000,0.0,200.0,atamabekleyenzz,No
2997,760235343966863360,Emrah İNCİ,memrahinci,Istanbul - Bayburt,Researcher | Middle East | Political Science |...,5863,5905,71,0.498215,False,False,1029,0.040000,36.0,200.0,memrahinci,No
2998,1553973684100124672,Murat Kkk,muratkkk18,,Normal sıradan bir insanım,1,10,26,0.090909,False,False,18,0.769231,0.0,13.0,muratkkk18,No


In [73]:
trainingUserDf.isBot.value_counts()

No     2424
Yes     576
Name: isBot, dtype: int64

### 2.2.2. Separate X and y values
We use only 4 features here to create a baseline model. However, it is not enough to get good results.

In [74]:
dfBotAll_train.head()

,user_id,user_name,user_screen_name,user_location,user_description,user_followers_count,user_friends_count,description_len,followers_to_all_ratio,user_is_verified,user_has_default_photo,user_statuses_count,retweet_total_ratio,num_median_favorites,num_of_tweets,screen_name,isBot
0,1512081815292432394,sezgin,sezgin953116371,,,46,430,0,0.096639,False,True,1535,0.050251,0.0,199.0,sezgin953116371,No
1,1425452291428077571,Adem Koç,gogoadem61,,,14,171,0,0.075676,False,False,113,0.761062,0.0,113.0,gogoadem61,No
2,328164303,Necmettin Balıkçı,dewil511,,,21,49,0,0.300000,False,False,219,0.010101,0.0,198.0,dewil511,Yes
3,1343666971368431622,Night Bird⁷🦉,midnight__bird,,"La vie est un sommeil, l’amour en est le rêve...",422,260,48,0.618768,False,False,15191,0.085000,1.0,200.0,midnight__bird,No
4,1240932880488038400,Samed Pınarcı,samedpinarci,,Orman Mühendisi - Orman İşletme Şefi - Orman G...,133,202,60,0.397015,False,False,3734,0.780000,0.0,200.0,samedpinarci,No


In [75]:
X = dfBotAll_train[['description_len', 'followers_to_all_ratio', 'retweet_total_ratio', 'num_median_favorites','user_friends_count','user_statuses_count','user_is_verified','user_has_default_photo']]

y = dfBotAll_train.isBot.apply(lambda x: 1 if x=='Yes' else 0)

### 2.2.3. Train-test split

In [76]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=42)

### 2.2.4. Train the model

In [98]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import  mean_squared_error, make_scorer

mse = make_scorer(mean_squared_error, greater_is_better=False)



parameters = { 
    'learning_rate': [0.2, 0.3, 0.4],
    'max_depth': [4,6,8],
    'n_estimators': [100,150],
    'min_child_weight':[1,3, 5,7],
    'subsample':[0.6, 0.7, 0.8, 1],
    'colsample_bytree':[0.4,0.6,0.8,1],
    'gamma':[0.3,0.4,0.5],
    'reg_lambda': [0.6,1]
    }


estimator = xgb.XGBRegressor(early_stopping_rounds=10, objective='binary:logistic')

botModel1 = GridSearchCV(estimator, parameters, n_jobs=4, 
                   cv=5, 
                   scoring=mse,
                   verbose=2, refit=True)

botModel1.fit(X_train, y_train)


Fitting 5 folds for each of 6912 candidates, totalling 34560 fits


GridSearchCV(cv=5,
             estimator=XGBRegressor(early_stopping_rounds=10,
                                    objective='binary:logistic'),
             n_jobs=4,
             param_grid={'colsample_bytree': [0.4, 0.6, 0.8, 1],
                         'gamma': [0.3, 0.4, 0.5],
                         'learning_rate': [0.2, 0.3, 0.4],
                         'max_depth': [4, 6, 8],
                         'min_child_weight': [1, 3, 5, 7],
                         'n_estimators': [100, 150], 'reg_lambda': [0.6, 1],
                         'subsample': [0.6, 0.7, 0.8, 1]},
             scoring=make_scorer(mean_squared_error, greater_is_better=False),
             verbose=2)

###Displaying the MSE

In [99]:
# make predictions
predictions1 = botModel1.predict(X_valid)

# evaluate on validation set
mse_bot1 = mean_squared_error(y_valid, predictions1)

print("MSE:", mse_bot1)




MSE: 0.12931427660179978


In [100]:
rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=None) 
rkf.get_n_splits(X, y)



50

In [ ]:
for i, (train_index, test_index) in enumerate(rkf.split(X)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")

# 3. MAKE PREDICTIONS

Here, you will make predictions with the models that you have trained above.

## 3.1. Predictions for Tweets (Political or Not)

In [102]:
# read the evaluation file as follows
evaluationTweetDf = pd.read_csv('{}evaluation-round3-tweet.csv'.format(DATA_PATH), dtype={0: str}, header=None, names=['tweet_id'])
evaluationTweetDf = evaluationTweetDf.dropna()
evaluationTweetDf

# merge it with the political dataframe so that you can use the make predictions based on the variables
dfPolitical_test = dfPolitical.merge(evaluationTweetDf)


# define X as we did above in section (2.x.2. Separate X and y values)

X = dfPolitical_test[['num_political_entities_tweet', 'num_political_entities_description', 'num_mentions', 'num_hashtags', 'total_interactions',  'political_word_ratio', 'has_link']]

# make predictions based on these variables
predictions_political = politicalModel1.predict(X)

In [103]:
#see the prediction of the tweets in another df to tune the model accordingly
dfyunus = dfPolitical_test
dfyunus = dfyunus[['tweet_id', 'user_description', 'text', 'num_mentions', 'num_hashtags','num_retweets', 'num_political_entities_tweet', 'num_political_entities_description', 'has_link']]
dfyunus['predictions']=predictions_political.tolist()
dfyunus

<ipython-input-103-4e3e58e770cb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfyunus['predictions']=predictions_political.tolist()


,tweet_id,user_description,text,num_mentions,num_hashtags,num_retweets,num_political_entities_tweet,num_political_entities_description,has_link,predictions
0,1434787703783051264,Ben konuşamam my time - fan account,@vmintr_vmin Annem bu evde yaşamaz o kadar cam...,1,0,0,0,0,False,0.153429
1,1367571642604544000,,@BarisTahmaz @MuratKeskin75 @emreakduz Koptum ...,3,0,0,0,0,False,0.118541
2,1589993032975544320,,@ATuncayOzkan Burada ilkesiz olan aslında AKP ...,1,0,0,4,0,True,0.943053
3,1565312596135354373,,@RTErdogan Yanındayız reis,1,0,0,3,0,False,0.869772
4,1579558096833511424,,@NureddinNebati #sondakika Sayın @RTErdogan cu...,2,1,0,5,0,True,0.932601
...,...,...,...,...,...,...,...,...,...,...
4995,1568250903358046212,"I'm Superman with the wind in his bag, she's L...",@madyaglizencisi @kilicdarogluk Şimdiye kadar ...,2,0,8,3,0,True,0.827013
4996,1595117878302236673,ZM,@meral_aksener Tarıma verilen değer ülkeye ver...,2,1,0,4,0,False,0.931820
4997,1366129358197444611,işsizim,@kpopgo6 Aynı zamanda diğer üyelerde,1,0,0,0,0,False,0.153429
4998,1347719781546303489,aka Mustasekittens 🐱 | artist | alt: @squished...,@RahulKohli13 SDFDGHDGFHGFJHGKJKDFHXDFHDFSDGS,1,0,0,0,0,False,0.105560


In [104]:
#predictions_political  = np.round(predictions_political).astype(np.int)
predictions_political

array([0.15342939, 0.11854111, 0.94305265, ..., 0.15342939, 0.10556009,
       0.9308454 ], dtype=float32)

### This part is important! We expect you to return your predictions in the following format:

In [105]:
modelPredTweet = dict([(x,float(y)) for x,y in zip([*dfPolitical_test.tweet_id], predictions_political)])
modelPredTweet

{'1434787703783051264': 0.15342938899993896,
 '1367571642604544000': 0.11854110658168793,
 '1589993032975544320': 0.9430526494979858,
 '1565312596135354373': 0.8697723150253296,
 '1579558096833511424': 0.9326013922691345,
 '1439547067337256967': 0.09914979338645935,
 '1559963768372740098': 0.924950897693634,
 '1562853131251118081': 0.7667136788368225,
 '1586021183958704128': 0.7286613583564758,
 '1585766233491886081': 0.8566349744796753,
 '1427746815420604417': 0.3136819303035736,
 '1352635736537882629': 0.48233097791671753,
 '1415032260571680768': 0.15342938899993896,
 '1548636597628899328': 0.9305713176727295,
 '1564926450096013313': 0.4447924792766571,
 '1585634359612420101': 0.9362574219703674,
 '1597138789108895744': 0.8813046216964722,
 '1391681495622995971': 0.056798968464136124,
 '1389951943343316995': 0.10556009411811829,
 '1452348722810138646': 0.7861003279685974,
 '1595829502021623812': 0.8787069916725159,
 '1413108476348354562': 0.10556009411811829,
 '1579408398894137344': 

## 3.2. Predictions for Users (Bot or Not)

In [106]:
evaluationUserDf = pd.read_csv('{}evaluation-round3-user.csv'.format(DATA_PATH), dtype={0: str}, header=None, names=['user_screen_name'])
evaluationUserDf = evaluationUserDf.dropna()

# merge it with the political dataframe so that you can use the make predictions based on the variables
dfBot_test = dfBotAll.merge(evaluationUserDf)

# define X as we did above in section (2.x.2. Separate X and y values)
X = dfBot_test[['description_len', 'followers_to_all_ratio', 'retweet_total_ratio', 'num_median_favorites','user_friends_count','user_statuses_count','user_is_verified','user_has_default_photo']]
# make predictions based on these variables
predictions_bot = botModel1.predict(X)

In [107]:
#our predictions will be float type such as 0.7 0.8 or 0.5 we are rounding them to 0 or 1 to classify them as bot or not
#predictions_bot  = np.round(predictions_bot).astype(np.int)
predictions_bot

array([0.48753294, 0.02769596, 0.15774131, ..., 0.5262369 , 0.01045625,
       0.01888994], dtype=float32)

In [108]:
modelPredUser = dict([(x,float(y)) for x,y in zip([*dfBot_test.user_screen_name], predictions_bot)])
modelPredUser

{'biologselim': 0.48753294348716736,
 'omerakdag34': 0.02769595943391323,
 'bilgin21604923': 0.15774130821228027,
 '_sydneycarton_': 0.14744976162910461,
 'denizlihabercom': 0.05733421817421913,
 'burakerbaychp': 0.010628607124090195,
 'mvnez': 0.18028853833675385,
 'qara118': 0.008328532800078392,
 'nabiyonyevrum': 0.02050877921283245,
 'farukhalit2': 0.06011510640382767,
 'harlunoshi': 0.0624602846801281,
 'heritagepaix': 0.2605848014354706,
 'nuranwolf': 0.036877721548080444,
 'politikgundem': 0.12396026402711868,
 'isakethudax': 0.00538578862324357,
 'enveraysevera': 0.031634312123060226,
 'ilaydejaneiro': 0.08113396167755127,
 '1905anason': 0.44532299041748047,
 'eraydurgut03': 0.06921222805976868,
 'dasiskein': 0.12133824825286865,
 'ercan_bas29': 0.10756717622280121,
 'mett_1907': 0.0667184367775917,
 'ondemir066': 0.36224365234375,
 'semihyeteer': 0.08719361573457718,
 'haberinyokcokk': 0.2992898225784302,
 'meleky_ozaydin': 0.2724059522151947,
 'mehmetaltay64': 0.0247978791594

# PREPARE SUBMISSION

You will need to submit exact same file produced by using the following code. Any deviation from the desired format willbe marked as 0.

In [109]:
# Explain your approach

data_explanations = '''


For the dataside we used the default values given to us as they seems to work pretty well
'''

feature_explanations = '''
For the detecting tweet (dfpolitical)
 In feature extraction part we have used the given features such as total interactions and num retweets. 
 In additin to that we have created a feature called political word ratio to see the political words this user has posted.

 ROUND-3
  We have added many relevant keyword to number of political entities function to classify the political tweets.
  We also added check_link function check whether tweeter link added. These links generally contains political mentions.
  

For the detecting userbot (dfbot) 
  We have selected the some of the features given to us such as is user verified or is he/she has default photo.
'''
model_explanations = '''
We have tried many models such as decision tree adaboost and neural network hovewe we have found the best result by using XGBoost.
One of the main advantages of using XGBoost over other models is its efficiency. 
It's been optimized to run faster than other implementations of gradient boosting, and it can handle large datasets with ease.
'''

additional_explanations = '''
For this round as  i said earlier we have tried many models which made us realize which model worked better for this occasion.
'''


In [110]:
predictions = {
    'round': ROUND,
    'student_id': STUDENT_ID_1,
    'user_predictions': modelPredUser,
    'tweet_predictions': modelPredTweet,
    'explanations': {
        'data': data_explanations,
        'feature': feature_explanations,
        'model': model_explanations,
        'other': additional_explanations,
    }
}


with open('predictions-{}_round{}.json'.format(STUDENT_ID_1, ROUND), 'w') as fl:
    fl.write(json.dumps(predictions, indent=4))

In [111]:
# Test your submission file

submission = json.load(open('predictions-{}_round{}.json'.format(STUDENT_ID_1 , ROUND), 'r'))
submission

{'round': 3,
 'student_id': '26617',
 'user_predictions': {'biologselim': 0.48753294348716736,
  'omerakdag34': 0.02769595943391323,
  'bilgin21604923': 0.15774130821228027,
  '_sydneycarton_': 0.14744976162910461,
  'denizlihabercom': 0.05733421817421913,
  'burakerbaychp': 0.010628607124090195,
  'mvnez': 0.18028853833675385,
  'qara118': 0.008328532800078392,
  'nabiyonyevrum': 0.02050877921283245,
  'farukhalit2': 0.06011510640382767,
  'harlunoshi': 0.0624602846801281,
  'heritagepaix': 0.2605848014354706,
  'nuranwolf': 0.036877721548080444,
  'politikgundem': 0.12396026402711868,
  'isakethudax': 0.00538578862324357,
  'enveraysevera': 0.031634312123060226,
  'ilaydejaneiro': 0.08113396167755127,
  '1905anason': 0.44532299041748047,
  'eraydurgut03': 0.06921222805976868,
  'dasiskein': 0.12133824825286865,
  'ercan_bas29': 0.10756717622280121,
  'mett_1907': 0.0667184367775917,
  'ondemir066': 0.36224365234375,
  'semihyeteer': 0.08719361573457718,
  'haberinyokcokk': 0.29928982